In [102]:
import kagglehub
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, f1_score

In [173]:
path = kagglehub.dataset_download("parisrohan/credit-score-classification")

print("Path to dataset files:", path)

df = pd.read_csv(path + "/train.csv")

Path to dataset files: /kaggle/input/credit-score-classification


<ipython-input-173-c8cad3162309>:5: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "/train.csv")


In [198]:
pd.set_option('display.max_columns', None)
df.tail()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",23,7,11.5,3.0,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,Poor
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",18,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,Poor
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,5729,2,"Auto Loan, and Student Loan",27,6,11.5,3.0,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,Poor
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",20,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.164979,Standard
99999,0x25fed,CUS_0x942c,August,Nicks,25,078-73-5990,Mechanic,39628.99_,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",18,6,11.5,3.0,Good,502.38,34.192463,31 Years and 10 Months,No,35.104023,167.1638651610451,!@9#%8,393.673696,Poor


In [174]:
print(df.isna().sum())

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64


In [192]:
#df.info()
df_copy = df.copy()

In [193]:
num_cols = ['Age','Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Monthly_Balance']
for col in num_cols:
  df_copy[col] = pd.to_numeric(df_copy[col], errors = 'coerce')
df_copy.isna().sum()

,0
ID,0
Customer_ID,0
Month,0
Name,9985
Age,4939
SSN,0
Occupation,0
Annual_Income,6980
Monthly_Inhand_Salary,15002
Num_Bank_Accounts,0


In [201]:
df_copy = df_copy.fillna(method='ffill')
df_copy.isna().sum()

<ipython-input-201-1ad55ff6359d>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_copy = df_copy.fillna(method='ffill')


,0
ID,0
Customer_ID,0
Month,0
Name,0
Age,0
SSN,0
Occupation,0
Annual_Income,0
Monthly_Inhand_Salary,0
Num_Bank_Accounts,0


In [212]:
plt.figure()

for col in num_cols:
  print(df_copy[col].agg(['min', 'max']))

min    -500.0
max    8698.0
Name: Age, dtype: float64
min        7005.93
max    24198062.00
Name: Annual_Income, dtype: float64
min      303.645417
max    15204.633333
Name: Monthly_Inhand_Salary, dtype: float64
min      -1
max    1798
Name: Num_Bank_Accounts, dtype: int64
min       0
max    1499
Name: Num_Credit_Card, dtype: int64
min       1
max    5797
Name: Interest_Rate, dtype: int64
min    -100.0
max    1496.0
Name: Num_of_Loan, dtype: float64
min    -5
max    67
Name: Delay_from_due_date, dtype: int64
min      -3.0
max    4397.0
Name: Num_of_Delayed_Payment, dtype: float64
min    -6.49
max    36.97
Name: Changed_Credit_Limit, dtype: float64
min       0.0
max    2597.0
Name: Num_Credit_Inquiries, dtype: float64
min       0.23
max    4998.07
Name: Outstanding_Debt, dtype: float64
min    20.0
max    50.0
Name: Credit_Utilization_Ratio, dtype: float64
min        0.0
max    82331.0
Name: Total_EMI_per_month, dtype: float64
min       0.000000
max    1977.326102
Name: Amount_invested_m

<Figure size 640x480 with 0 Axes>

In [220]:
# Replace values in 'Age' column that are over 100 or under 0 with the median value
df_copy['Age'] = df_copy['Age'].apply(lambda x: df_copy['Age'].median() if x > 100 or x < 0 else x)
df_copy['']

# Check the min/max of Age again to confirm the change
print(df_copy['Age'].agg(['min', 'max']))


min     14.0
max    100.0
Name: Age, dtype: float64
